In [3]:
import json 
import numpy as np
import matplotlib.pyplot as plt
import random
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch.nn.functional as F

from scipy.spatial import distance
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

import time
import os
import math
from tqdm import tqdm

# import simplecnn
import pickle
import argparse
import scipy
import copy

# edit scenefile - vbsl 151 (test images)

In [ ]:
# Random generation (151 test images, grayscale)

_path = '20240924_Var6vbsl_im151_AppleMesh00_neutral.json'
original_scenefile = json.load(open(_path, 'r'))

# define a new scene file by copying the original scene file
new_scenefile = copy.deepcopy(original_scenefile)

In [ ]:
# then edit scene file parameters as follows:

new_scenefile['OBJECTS']['AppleMesh00']['meshpath'] = '/mkturkfiles/scenebags/objectome3d/face/AppleMesh00_neutral.glb' # face mesh path
new_scenefile['durationMS'] = [9] # duration (in ms)

# --------------- Generate random sizes ---------------
size_range = (min(original_scenefile['OBJECTS']['AppleMesh00']['sizeTHREEJS']), max(original_scenefile['OBJECTS']['AppleMesh00']['sizeTHREEJS']))
new_scenefile['OBJECTS']['AppleMesh00']['sizeTHREEJS'] = [random.uniform(*size_range) for _ in range(151)]

# --------------- Generate random rotations ---------------
rotation_x_range = (min(original_scenefile['OBJECTS']['AppleMesh00']['rotationDegrees']['x']), max(original_scenefile['OBJECTS']['AppleMesh00']['rotationDegrees']['x']))
new_scenefile['OBJECTS']['AppleMesh00']['rotationDegrees']['x'] = [random.uniform(*rotation_x_range) for _ in range(5000)]

rotation_y_range = (min(original_scenefile['OBJECTS']['AppleMesh00']['rotationDegrees']['y']), max(original_scenefile['OBJECTS']['AppleMesh00']['rotationDegrees']['y']))
new_scenefile['OBJECTS']['AppleMesh00']['rotationDegrees']['y'] = [random.uniform(*rotation_y_range) for _ in range(5000)]

new_scenefile['OBJECTS']['AppleMesh00']['positionTHREEJS']['x'] = [0.0] * 151
new_scenefile['OBJECTS']['AppleMesh00']['positionTHREEJS']['y'] = [0.0] * 151
new_scenefile['OBJECTS']['AppleMesh00']['positionTHREEJS']['z'] = [0.0] * 151

# --------------- Generate random light positions ---------------
light_x_range = (min(original_scenefile['LIGHTS']['light00']['position']['x']), max(original_scenefile['LIGHTS']['light00']['position']['x']))
new_scenefile['LIGHTS']['light00']['position']['x'] = [random.uniform(*light_x_range) for _ in range(5000)]
light_y_range = (min(original_scenefile['LIGHTS']['light00']['position']['y']), max(original_scenefile['LIGHTS']['light00']['position']['y']))
new_scenefile['LIGHTS']['light00']['position']['y'] = [random.uniform(*light_y_range) for _ in range(5000)]
light_z_range = (min(original_scenefile['LIGHTS']['light00']['position']['z']), max(original_scenefile['LIGHTS']['light00']['position']['z']))
new_scenefile['LIGHTS']['light00']['position']['z'] = [random.uniform(*light_z_range) for _ in range(5000)]

# --------------- Shuffle and assign background indices ---------------
random.shuffle(original_scenefile['IMAGES']['imageidx'])

grayscaled = True
if grayscaled : # if want grayscaled images
    new_scenefile['IMAGES']['imagebag'] = '/mkturkfiles/scenebags/objectome3d/background_im_flipped/'
else : # if want color images
    new_scenefile['IMAGES']['imagebag'] = '/mkturkfiles/scenebags/objectome3d/background_im_flipped_color/'
    if 'grayscale' in new_scenefile.get('OBJECTFILTERS', {}):
        del new_scenefile['OBJECTFILTERS']['grayscale']

image_idx_list = set(original_scenefile['IMAGES']['imageidx'])
image_idx_list = list(image_idx_list)
new_scenefile['IMAGES']['imageidx'] = random.choices(image_idx_list, k=151)

# Save the new scene file
new_path = f'20250605_Var6vbsl_im151_AppleMesh00_neutral.json'
with open(new_path, 'w') as f:
    json.dump(new_scenefile, f)
print("'"+new_path+"',")

# edit scenefile - vbsl 50k (train images)

In [8]:
# Random generation (50000 train images, grayscale)

_path2 = '20241004_Var6vbsl_im50000_younah_neutral.json'
original_scenefile = json.load(open(_path2, 'r'))

# define a new scene file by copying the original scene file
new_scenefile = copy.deepcopy(original_scenefile)

In [ ]:
# then edit scene file parameters as follows:

new_scenefile['OBJECTS']['AppleMesh00']['meshpath'] = '/mkturkfiles/scenebags/objectome3d/face/AppleMesh00_neutral_texture.glb' # face mesh path
new_scenefile['durationMS'] = [9] # duration 

# --------------- Generate random sizes ---------------
random.shuffle(original_scenefile['OBJECTS']['AppleMesh00']['sizeTHREEJS'])
new_scenefile['OBJECTS']['AppleMesh00']['sizeTHREEJS'] = original_scenefile['OBJECTS']['AppleMesh00']['sizeTHREEJS']

# --------------- Generate random rotations ---------------
random.shuffle(original_scenefile['OBJECTS']['AppleMesh00']['rotationDegrees']['x'] )
random.shuffle(original_scenefile['OBJECTS']['AppleMesh00']['rotationDegrees']['y'] )
# random.shuffle(original_scenefile['OBJECTS']['AppleMesh00']['rotationDegrees']['z'] ) <- don't edit z-axis rotation

new_scenefile['OBJECTS']['AppleMesh00']['rotationDegrees']['x'] = original_scenefile['OBJECTS']['AppleMesh00']['rotationDegrees']['x'] 
new_scenefile['OBJECTS']['AppleMesh00']['rotationDegrees']['y'] = original_scenefile['OBJECTS']['AppleMesh00']['rotationDegrees']['y'] 
# new_scenefile['OBJECTS']['AppleMesh00']['rotationDegrees']['z'] = original_scenefile['OBJECTS']['AppleMesh00']['rotationDegrees']['z'] 

# --------------- Generate random light positions ---------------
random.shuffle(original_scenefile['LIGHTS']['light00']['position']['x'])
random.shuffle(original_scenefile['LIGHTS']['light00']['position']['y'])
random.shuffle(original_scenefile['LIGHTS']['light00']['position']['z'])

new_scenefile['LIGHTS']['light00']['position']['x'] = original_scenefile['LIGHTS']['light00']['position']['x']
new_scenefile['LIGHTS']['light00']['position']['y'] = original_scenefile['LIGHTS']['light00']['position']['y']
new_scenefile['LIGHTS']['light00']['position']['z'] = original_scenefile['LIGHTS']['light00']['position']['z']

# --------------- Shuffle and assign background indices ---------------
random.shuffle(original_scenefile['IMAGES']['imageidx'])

grayscaled = True
if grayscaled : # if want grayscaled images
    new_scenefile['IMAGES']['imagebag'] = '/mkturkfiles/scenebags/objectome3d/background_im_flipped/'
else : # if want color images
    new_scenefile['IMAGES']['imagebag'] = '/mkturkfiles/scenebags/objectome3d/background_im_flipped_color/'
    if 'grayscale' in new_scenefile.get('OBJECTFILTERS', {}):
        del new_scenefile['OBJECTFILTERS']['grayscale']

image_idx_list = set(original_scenefile['IMAGES']['imageidx'])
image_idx_list = list(image_idx_list)
new_scenefile['IMAGES']['imageidx'] = random.choices(image_idx_list, k=50000)

# Save the new scene file
_path = f'20250114_Var6vbsl_im50000_younah_neutral_texture.json'
with open(_path, 'w') as f:
    json.dump(new_scenefile, f)


# edit SceneFile - vbsl 50k(or less) basel

In [ ]:
_path = './20220603_Var6vbsl_set1_im50000_AppleMesh00_dur400ms.json'
AppleMesh00_scenefile = json.load(open(_path, 'r'))

# _path = './20220603_Var6vbsl_set1_im50000_AppleMesh01_dur400ms.json'
# AppleMesh01_scenefile = json.load(open(_path, 'r'))labusag

In [ ]:
np.min(AppleMesh00_scenefile['LIGHTS']['light00']['position']['y'])

In [ ]:
np.max(AppleMesh00_scenefile['IMAGES']['imageidx']), len(set(AppleMesh00_scenefile['IMAGES']['imageidx']))

In [ ]:
import copy
new_scenefile_0 = copy.deepcopy(AppleMesh00_scenefile)

In [ ]:
def create_new_scenefile(scenefile, obj_name, meshpath, sample_size) :
    # obj name
    scenefile['OBJECTS'] = {}
    scenefile['OBJECTS'][obj_name] = AppleMesh00_scenefile['OBJECTS']['AppleMesh00']
    # meshpath
    scenefile['OBJECTS'][obj_name]['meshpath'] = meshpath
    
    # size
    scenefile['OBJECTS'][obj_name]['sizeTHREEJS'] = list(np.random.uniform(0.2*1.61, 0.42*1.61, sample_size))
    
    # rotation
    scenefile['OBJECTS'][obj_name]['rotationDegrees']['x'] = list(np.random.uniform(-45, 45, sample_size))
    scenefile['OBJECTS'][obj_name]['rotationDegrees']['y'] = list(np.random.uniform(-90, 90, sample_size))
    scenefile['OBJECTS'][obj_name]['rotationDegrees']['z'] = scenefile['OBJECTS'][obj_name]['rotationDegrees']['z'][:sample_size]
    
    # background
    scenefile['IMAGES']['imageidx'] = list(random.choices(range(1428), k=sample_size)) # a total of 1428 bg
    print(f"covered bg: {len(set(scenefile['IMAGES']['imageidx']))}")
    scenefile['IMAGES']['imageidx'] = list(scenefile['IMAGES']['imageidx'])
    # cover_all_bg = False
    # while not cover_all_bg:
    #     scenefile['IMAGES']['imageidx'] = random.choices(np.arange(1428), k=sample_size) # a total of 1428 bg
    #     if len(set(scenefile['IMAGES']['imageidx'])) == 1428:
    #         cover_all_bg = True
    # assert(len(set(scenefile['IMAGES']['imageidx'])) == 1428)
    
    # lighting
    scenefile['LIGHTS']['light00']['position']['x'] = list(np.random.uniform(-1, 1, sample_size))
    scenefile['LIGHTS']['light00']['position']['y'] = list(np.random.uniform(-1, 1, sample_size))
    scenefile['LIGHTS']['light00']['position']['z'] = scenefile['LIGHTS']['light00']['position']['z'][:sample_size]
    
    # add grey scale filter 
    scenefile['OBJECTFILTERS']['grayscale'] = [100]
    # remove texture
    if 'notexture' in obj_name:
        scenefile['OBJECTS'][obj_name]['texture'] = 'false'
    # change duration 
    scenefile['durationMS'] = [18]
    
    # save scenefile
    _path = f'./vbsl50k_Basel/202305051_Var6vbsl_im{sample_size}_{obj_name}.json'
    with open(_path, 'w') as f:
        json.dump(scenefile, f)
    
    return scenefile

In [ ]:
sample_size = 5000
for basel_index in range(8):
    new_scenefile_0 = create_new_scenefile(new_scenefile_0, f"Baselmesh{basel_index}_notexture", 
                     f"/mkturkfiles/scenebags/objectome3d/Josh/Baselmesh{basel_index}.glb", sample_size)

# edit SceneFile - vbsl 50k dist

In [ ]:
_path = './20220603_Var6vbsl_set1_im50000_AppleMesh00_dur400ms.json'
AppleMesh00_scenefile = json.load(open(_path, 'r'))

_path = './20220603_Var6vbsl_set1_im50000_AppleMesh01_dur400ms.json'
AppleMesh01_scenefile = json.load(open(_path, 'r'))

In [ ]:
import copy
new_AppleMesh00_scenefile = copy.deepcopy(AppleMesh00_scenefile)
new_AppleMesh01_scenefile = copy.deepcopy(AppleMesh01_scenefile)

In [ ]:
from scipy.stats import truncnorm
def get_truncated_normal(mean=0, sd=1, low=0, upp=10):
    return truncnorm(
        (low - mean) / sd, (upp - mean) / sd, loc=mean, scale=sd)

In [ ]:
len(new_AppleMesh00_scenefile['OBJECTS']['AppleMesh00']['sizeTHREEJS'])

In [ ]:
# no need to change lighting
# # change lighting (range=[-1, 1])
# tmp_gen = get_truncated_normal(mean=0, sd=0.3, low=-1, upp=1)
# new_array = tmp_gen.rvs(50000)
# new_AppleMesh00_scenefile['LIGHTS']['light00']['position']['x'] = new_array.tolist()
# new_array = tmp_gen.rvs(50000)
# new_AppleMesh00_scenefile['LIGHTS']['light00']['position']['y'] = new_array.tolist()
# new_array = tmp_gen.rvs(50000)
# new_AppleMesh01_scenefile['LIGHTS']['light00']['position']['x'] = new_array.tolist()
# new_array = tmp_gen.rvs(50000)
# new_AppleMesh01_scenefile['LIGHTS']['light00']['position']['y'] = new_array.tolist()

In [ ]:
# change rotation (x_range=[-45, 45], y_range=[-90, 90])
tmp_gen = get_truncated_normal(mean=0, sd=13.5, low=-45, upp=45)
new_array = tmp_gen.rvs(50000)
new_AppleMesh00_scenefile['OBJECTS']['AppleMesh00']['rotationDegrees']['x'] = new_array.tolist()
new_array = tmp_gen.rvs(50000)
new_AppleMesh01_scenefile['OBJECTS']['AppleMesh01']['rotationDegrees']['x'] = new_array.tolist()

tmp_gen = get_truncated_normal(mean=0, sd=27, low=-90, upp=90)
new_array = tmp_gen.rvs(50000)
new_AppleMesh00_scenefile['OBJECTS']['AppleMesh00']['rotationDegrees']['y'] = new_array.tolist()
new_array = tmp_gen.rvs(50000)
new_AppleMesh01_scenefile['OBJECTS']['AppleMesh01']['rotationDegrees']['y'] = new_array.tolist()

In [ ]:
# change size (range=[0.2, 0.42])
from scipy.stats import expon
_scale = 0.055
img_size = 1 # inch(es)

# sample 
new_array = expon.rvs(scale=_scale, size=50000)
num_rej = (new_array>0.22).sum()
while num_rej!=0:
    new_array[new_array>0.22] = expon.rvs(scale=_scale, size=num_rej)
    num_rej = (new_array>0.22).sum()
new_array = -new_array + 0.42
# record
new_AppleMesh00_scenefile['OBJECTS']['AppleMesh00']['sizeTHREEJS'] = (new_array*img_size).tolist()

# sample 
new_array = expon.rvs(scale=_scale, size=50000)
num_rej = (new_array>0.22).sum()
while num_rej!=0:
    new_array[new_array>0.22] = expon.rvs(scale=_scale, size=num_rej)
    num_rej = (new_array>0.22).sum()
new_array = -new_array + 0.42
# record
new_AppleMesh01_scenefile['OBJECTS']['AppleMesh01']['sizeTHREEJS'] = (new_array*img_size).tolist()

In [ ]:
# change the size of the img (in inches)
new_AppleMesh00_scenefile['IMAGES']['sizeTHREEJS'][0] = img_size
new_AppleMesh01_scenefile['IMAGES']['sizeTHREEJS'][0] = img_size

In [ ]:
_path = './20230206_Var6vbsl_GaussianVSL_im50000_AppleMesh00_dur400ms.json'
with open(_path, 'w') as f:
    json.dump(new_AppleMesh00_scenefile, f)
    
_path = './20230206_Var6vbsl_GaussianVSL_im50000_AppleMesh01_dur400ms.json'
with open(_path, 'w') as f:
    json.dump(new_AppleMesh01_scenefile, f)

# i2 - create new SceneFile for 8IDs

In [ ]:
_path = './20230325_Var6vbsl_set0_im31_AppleMesh00_dur30ms_lab.json'
AppleMesh00_scenefile = json.load(open(_path, 'r'))

_path = './20230325_Var6vbsl_set0_im31_AppleMesh01_dur30ms_lab.json'
AppleMesh01_scenefile = json.load(open(_path, 'r'))

In [ ]:
# don't need to change: CAMERAS, LIGHTS, OBJECTFILTERS, IMAGEFILTERS, durationMS
# change: OBJECTS 
#         - object name, meshpath, objectdoc?, size
#         - rotationDegrees: x and y
# change: IMAGES
#         - imageidx
AppleMesh00_scenefile.keys()

In [ ]:
AppleMesh00_scenefile['OBJECTS']['AppleMesh00']['sizeInches'][:5]

In [ ]:
AppleMesh01_scenefile['OBJECTS']['AppleMesh01']['sizeInches'][:5]

In [ ]:
background_id_set = set(AppleMesh00_scenefile['IMAGES']['imageidx'])
background_id_set.remove("")
background_id_set = list(background_id_set)
background_id_set

In [ ]:
def create_new_scenefile(scenefile, obj_name, meshpath):
    # obj name
    scenefile['OBJECTS'] = {}
    scenefile['OBJECTS'][obj_name] = AppleMesh00_scenefile['OBJECTS']['AppleMesh00']
    # meshpath
    scenefile['OBJECTS'][obj_name]['meshpath'] = meshpath
    # objectdoc
    # scenefile['OBJECTS'][obj_name]['objectdoc'] = ?
    # size
    scenefile['OBJECTS'][obj_name]['sizeInches'] = list(np.random.uniform(0.2, 0.42, 31))
    scenefile['OBJECTS'][obj_name]['sizeInches'][30] = 0.417422222
    # rotation
    scenefile['OBJECTS'][obj_name]['rotationDegrees']['x'] = list(np.random.uniform(-45, 45, 31))
    scenefile['OBJECTS'][obj_name]['rotationDegrees']['x'][30] = 0
    scenefile['OBJECTS'][obj_name]['rotationDegrees']['y'] = list(np.random.uniform(-90, 90, 31))
    scenefile['OBJECTS'][obj_name]['rotationDegrees']['y'][30] = 0
    # background
    cover_all_bg = False
    while not cover_all_bg:
        scenefile['IMAGES']['imageidx'] = random.choices(background_id_set, k=31)
        scenefile['IMAGES']['imageidx'][30] = ""
        if len(set(scenefile['IMAGES']['imageidx'])) == 11:
            cover_all_bg = True
    assert(len(set(scenefile['IMAGES']['imageidx'])) == 11)
    # save scenefile
    _path = f'./20230325_Var6vbsl_set0_im31_{obj_name}_dur30ms_lab.json'
    with open(_path, 'w') as f:
        json.dump(scenefile, f)
    
    return scenefile

In [ ]:
import copy
praneeth_scenefile = copy.deepcopy(AppleMesh00_scenefile)
sophie_scenefile = copy.deepcopy(AppleMesh00_scenefile)

sophie_potato_scenefile = copy.deepcopy(AppleMesh00_scenefile)
sophie_eye_over_nose_scenefile = copy.deepcopy(AppleMesh00_scenefile)
sophie_nose_under_eye_scenefile = copy.deepcopy(AppleMesh00_scenefile)
sophie_thatcherized_scenefile = copy.deepcopy(AppleMesh00_scenefile)

In [ ]:
praneeth_scenefile = create_new_scenefile(praneeth_scenefile, "praneeth", 
                     "/mkturkfiles/scenebags/objectome3d/face/praneethapplefacepotatoheadnormalhires.glb")
sophie_scenefile = create_new_scenefile(sophie_scenefile, "sophie", 
                     "/mkturkfiles/scenebags/objectome3d/face/sophieapplefacepotatoheadnormalhires.glb")

In [ ]:
sophie_potato_scenefile = create_new_scenefile(sophie_potato_scenefile, "sophie_potato", 
                     "/mkturkfiles/scenebags/FaceStimuli/finished/sophie_Nose_0_Mouth_0_EyeL_22_5_EyeR_-22_5.glb")
sophie_thatcherized_scenefile = create_new_scenefile(sophie_thatcherized_scenefile, "sophie_thatcherized", 
                     "/mkturkfiles/scenebags/FaceStimuli/finished/thatcherizedsophieparts.glb")
sophie_eye_over_nose_scenefile = create_new_scenefile(sophie_eye_over_nose_scenefile, "sophie_eye_over_nose", 
                     "/mkturkfiles/scenebags/FaceStimuli/finished/sophie_Nose_0_Mouth_0_EyeL_22_5_EyeR_0.glb")
sophie_nose_under_eye_scenefile = create_new_scenefile(sophie_nose_under_eye_scenefile, "sophie_nose_under_eye", 
                     "/mkturkfiles/scenebags/FaceStimuli/finished/sophie_Nose_-22_5_Mouth_0_EyeL_22_5_EyeR_-22_5.glb")

# i2 - token File

In [ ]:
_path = './i2_scenefiles/20220202_token_AppleMesh00.json'
AppleMesh00_scenefile = json.load(open(_path, 'r'))

In [ ]:
AppleMesh00_scenefile.keys()

In [ ]:
import copy
AppleMesh01_scenefile = copy.deepcopy(AppleMesh00_scenefile)
praneeth_scenefile = copy.deepcopy(AppleMesh00_scenefile)
sophie_scenefile = copy.deepcopy(AppleMesh00_scenefile)

sophie_potato_scenefile = copy.deepcopy(AppleMesh00_scenefile)
sophie_eye_over_nose_scenefile = copy.deepcopy(AppleMesh00_scenefile)
sophie_nose_under_eye_scenefile = copy.deepcopy(AppleMesh00_scenefile)
sophie_thatcherized_scenefile = copy.deepcopy(AppleMesh00_scenefile)

In [ ]:
def create_new_token_file(scenefile, obj_name, meshpath):
    # obj name
    scenefile['OBJECTS'] = {}
    scenefile['OBJECTS'][obj_name] = AppleMesh00_scenefile['OBJECTS']['AppleMesh00']
    # meshpath
    scenefile['OBJECTS'][obj_name]['meshpath'] = meshpath
    
    # save scenefile
    _path = f'./i2_scenefiles/20220202_token_{obj_name}.json'
    with open(_path, 'w') as f:
        json.dump(scenefile, f)
    
    return scenefile

In [ ]:
AppleMesh01_scenefile = create_new_token_file(AppleMesh01_scenefile, "AppleMesh01", 
                     "/mkturkfiles/scenebags/objectome3d/face/AppleMesh01applefacepotatoheadnormalhires.glb")
praneeth_scenefile = create_new_token_file(praneeth_scenefile, "praneeth", 
                     "/mkturkfiles/scenebags/objectome3d/face/praneethapplefacepotatoheadnormalhires.glb")
sophie_scenefile = create_new_token_file(sophie_scenefile, "sophie", 
                     "/mkturkfiles/scenebags/objectome3d/face/sophieapplefacepotatoheadnormalhires.glb")

sophie_potato_scenefile = create_new_token_file(sophie_potato_scenefile, "sophie_potato", 
                     "/mkturkfiles/scenebags/FaceStimuli/finished/sophie_Nose_0_Mouth_0_EyeL_22_5_EyeR_-22_5.glb")
sophie_thatcherized_scenefile = create_new_token_file(sophie_thatcherized_scenefile, "sophie_thatcherized", 
                     "/mkturkfiles/scenebags/FaceStimuli/finished/thatcherizedsophieparts.glb")
sophie_eye_over_nose_scenefile = create_new_token_file(sophie_eye_over_nose_scenefile, "sophie_eye_over_nose", 
                     "/mkturkfiles/scenebags/FaceStimuli/finished/sophie_Nose_0_Mouth_0_EyeL_22_5_EyeR_0.glb")
sophie_nose_under_eye_scenefile = create_new_token_file(sophie_nose_under_eye_scenefile, "sophie_nose_under_eye", 
                     "/mkturkfiles/scenebags/FaceStimuli/finished/sophie_Nose_-22_5_Mouth_0_EyeL_22_5_EyeR_-22_5.glb")

In [ ]:
AppleMesh01_scenefile = create_new_token_file(praneeth_scenefile, "Basel450", 
                      "/mkturkfiles/scenebags/objectome3d/Josh/Baselmesh450.glb")
AppleMesh01_scenefile = create_new_token_file(sophie_scenefile, "Basel29", 
                      "/mkturkfiles/scenebags/objectome3d/Josh/Baselmesh29.glb")
AppleMesh01_scenefile = create_new_token_file(sophie_potato_scenefile, "Basel231", 
                      "/mkturkfiles/scenebags/objectome3d/Josh/Baselmesh231.glb")

In [ ]:
AppleMesh01_scenefile = create_new_token_file(praneeth_scenefile, "Basel450", 
                      "/mkturkfiles/scenebags/objectome3d/Josh/Baselmesh450.glb")
AppleMesh01_scenefile = create_new_token_file(sophie_scenefile, "Basel29", 
                      "/mkturkfiles/scenebags/objectome3d/Josh/Baselmesh29.glb")
AppleMesh01_scenefile = create_new_token_file(sophie_potato_scenefile, "Basel231", 
                      "/mkturkfiles/scenebags/objectome3d/Josh/Baselmesh231.glb")

In [ ]:
AppleMesh01_scenefile = create_new_token_file(sophie_potato_scenefile, "Baselmesh_ood_0", 
                      "/mkturkfiles/scenebags/objectome3d/Josh/Baselmesh_ood_0.glb")
AppleMesh01_scenefile = create_new_token_file(sophie_scenefile, "Baselmesh_ood_1", 
                      "/mkturkfiles/scenebags/objectome3d/Josh/Baselmesh_ood_1.glb")

# Basel OOD vbsli*151

In [ ]:
_path = './20211011_Var6vbsl_set0_im151_AppleMesh00_dur200ms_lab_updated.json'
AppleMesh00_scenefile = json.load(open(_path, 'r'))

_path = './20211011_Var6vbsl_set0_im151_AppleMesh01_dur200ms_lab_updated.json'
AppleMesh01_scenefile = json.load(open(_path, 'r'))

In [ ]:
import copy
sophie_scenefile = copy.deepcopy(AppleMesh00_scenefile)
praneeth_scenefile = copy.deepcopy(AppleMesh01_scenefile)

basel_odd_0_scenefile = copy.deepcopy(AppleMesh00_scenefile)
basel_odd_1_scenefile = copy.deepcopy(AppleMesh01_scenefile)

basel_odd_0_notexture_scenefile = copy.deepcopy(AppleMesh00_scenefile)
basel_odd_1_notexture_scenefile = copy.deepcopy(AppleMesh01_scenefile)

In [ ]:
AppleMesh00_scenefile.keys()

In [ ]:
def create_new_scenefile(scenefile, obj_name, meshpath):
    # obj name
    scenefile['OBJECTS'] = {}
    scenefile['OBJECTS'][obj_name] = AppleMesh00_scenefile['OBJECTS']['AppleMesh00']
    # meshpath
    scenefile['OBJECTS'][obj_name]['meshpath'] = meshpath
    
    if 'basel' in obj_name.lower():
        scenefile['OBJECTFILTERS']['grayscale'] = [100]
    if 'notexture' in obj_name:
        scenefile['OBJECTS'][obj_name]['texture'] = 'false'
    
    scenefile['durationMS'] = [18]
    
    # save scenefile
    _path = f'./20230406_Var6vbsl_set0_im151_{obj_name}_dur200ms_lab.json'
    with open(_path, 'w') as f:
        json.dump(scenefile, f)
    
    return scenefile

In [ ]:
sophie_scenefile = create_new_scenefile(sophie_scenefile, 'sophie', 
                                       "/mkturkfiles/scenebags/objectome3d/face/sophieapplefacepotatoheadnormalhires.glb")
praneeth_scenefile = create_new_scenefile(praneeth_scenefile, 'praneeth', 
                                       "/mkturkfiles/scenebags/objectome3d/face/praneethapplefacepotatoheadnormalhires.glb")

In [ ]:
basel_odd_0_scenefile = create_new_scenefile(basel_odd_0_scenefile, 'Baselmesh_ood_0', 
                                       "/mkturkfiles/scenebags/objectome3d/Josh/Baselmesh_ood_0.glb")
basel_odd_1_scenefile = create_new_scenefile(basel_odd_1_scenefile, 'Baselmesh_ood_1', 
                                       "/mkturkfiles/scenebags/objectome3d/Josh/Baselmesh_ood_1.glb")

In [ ]:
basel_odd_0_notexture_scenefile = create_new_scenefile(basel_odd_0_notexture_scenefile, 'Baselmesh_ood_0_notexture', 
                                       "/mkturkfiles/scenebags/objectome3d/Josh/Baselmesh_ood_0.glb")
basel_odd_1_notexture_scenefile = create_new_scenefile(basel_odd_1_notexture_scenefile, 'Baselmesh_ood_1_notexture', 
                                       "/mkturkfiles/scenebags/objectome3d/Josh/Baselmesh_ood_1.glb")

In [ ]:
basel_odd_0_notexture_scenefile = create_new_scenefile(basel_odd_0_notexture_scenefile, 'Basel231', 
                                       "/mkturkfiles/scenebags/objectome3d/Josh/Baselmesh231.glb")
basel_odd_1_notexture_scenefile = create_new_scenefile(basel_odd_1_notexture_scenefile, 'Basel474', 
                                       "/mkturkfiles/scenebags/objectome3d/Josh/Baselmesh474.glb")

# aug vbsli* 151 AppleMesh00-AppleMesh01; for ideal observer

In [ ]:
_path = './20211011_Var6vbsl_set0_im151_AppleMesh00_dur200ms_lab_updated.json'
AppleMesh00_scenefile = json.load(open(_path, 'r'))

_path = './20211011_Var6vbsl_set0_im151_AppleMesh01_dur200ms_lab_updated.json'
AppleMesh01_scenefile = json.load(open(_path, 'r'))

In [ ]:
# total_images = 100 images * 100 pairs * 2 = 20,000
num_samples = 100

import copy
AppleMesh00_scenefile_upright = copy.deepcopy(AppleMesh00_scenefile)
AppleMesh00_scenefile_IfAppleMesh00 = copy.deepcopy(AppleMesh00_scenefile)
AppleMesh00_scenefile_IfAppleMesh01 = copy.deepcopy(AppleMesh00_scenefile)

AppleMesh01_scenefile_upright = copy.deepcopy(AppleMesh01_scenefile)
AppleMesh01_scenefile_IfAppleMesh00 = copy.deepcopy(AppleMesh01_scenefile)
AppleMesh01_scenefile_IfAppleMesh01 = copy.deepcopy(AppleMesh01_scenefile)

In [ ]:
AppleMesh00_scenefile_upright['LIGHTS']['light00']['position']['y'] = AppleMesh00_scenefile['LIGHTS']['light00']['position']['y'][0:25] + \
                                                                AppleMesh00_scenefile['LIGHTS']['light00']['position']['y'][50:75]

AppleMesh00_scenefile_upright['IMAGES']['imageidx'] = AppleMesh00_scenefile['IMAGES']['imageidx'][0:25] + AppleMesh00_scenefile['IMAGES']['imageidx'][50:75]

AppleMesh00_scenefile_upright['OBJECTFILTERS']['invert'] = ['']

AppleMesh00_scenefile_upright['OBJECTS']["AppleMesh00"]['rotationDegrees']['z'] = AppleMesh00_scenefile['OBJECTS']["AppleMesh00"]['rotationDegrees']['z'][0:25] + \
                                                                        AppleMesh00_scenefile['OBJECTS']["AppleMesh00"]['rotationDegrees']['z'][50:75]
AppleMesh00_scenefile_upright['OBJECTS']["AppleMesh00"]['rotationDegrees']['x'] = AppleMesh00_scenefile['OBJECTS']["AppleMesh00"]['rotationDegrees']['x'][0:25] + \
                                                                        AppleMesh00_scenefile['OBJECTS']["AppleMesh00"]['rotationDegrees']['x'][50:75]
AppleMesh00_scenefile_upright['OBJECTS']["AppleMesh00"]['rotationDegrees']['y'] = AppleMesh00_scenefile['OBJECTS']["AppleMesh00"]['rotationDegrees']['y'][0:25] + \
                                                 
# save scenefile
_path = f'./20230421_vbsl151upright_AppleMesh00.json'
with open(_path, 'w') as f:
    json.dump(AppleMesh00_scenefile_upright, f)

In [ ]:
AppleMesh01_scenefile_upright['LIGHTS']['light00']['position']['y'] = AppleMesh01_scenefile['LIGHTS']['light00']['position']['y'][0:25] + \
                                                                AppleMesh01_scenefile['LIGHTS']['light00']['position']['y'][50:75]

AppleMesh01_scenefile_upright['IMAGES']['imageidx'] = AppleMesh01_scenefile['IMAGES']['imageidx'][0:25] + AppleMesh01_scenefile['IMAGES']['imageidx'][50:75]

AppleMesh01_scenefile_upright['OBJECTFILTERS']['invert'] = ['']

AppleMesh01_scenefile_upright['OBJECTS']["AppleMesh01"]['rotationDegrees']['z'] = AppleMesh01_scenefile['OBJECTS']["AppleMesh01"]['rotationDegrees']['z'][0:25] + \
                                                                        AppleMesh01_scenefile['OBJECTS']["AppleMesh01"]['rotationDegrees']['z'][50:75]
AppleMesh01_scenefile_upright['OBJECTS']["AppleMesh01"]['rotationDegrees']['x'] = AppleMesh01_scenefile['OBJECTS']["AppleMesh01"]['rotationDegrees']['x'][0:25] + \
                                                                        AppleMesh01_scenefile['OBJECTS']["AppleMesh01"]['rotationDegrees']['x'][50:75]
AppleMesh01_scenefile_upright['OBJECTS']["AppleMesh01"]['rotationDegrees']['y'] = AppleMesh01_scenefile['OBJECTS']["AppleMesh01"]['rotationDegrees']['y'][0:25] + \
                                                                        AppleMesh01_scenefile['OBJECTS']["AppleMesh01"]['rotationDegrees']['y'][50:75]

AppleMesh01_scenefile_upright['OBJECTS']['AppleMesh01']['sizeInches'] = AppleMesh01_scenefile['OBJECTS']['AppleMesh01']['sizeInches'][0:25] + \
                                                            AppleMesh01_scenefile['OBJECTS']['AppleMesh01']['sizeInches'][50:75]

# save scenefile
_path = f'./20230421_vbsl151upright_AppleMesh01.json'
with open(_path, 'w') as f:
    json.dump(AppleMesh01_scenefile_upright, f)

In [ ]:
def add_noise(scenefile, obj_name, meshpath):
    ## deal with diff obj_name
    ori_obj_name = list(scenefile['OBJECTS'].keys())[0]
    if obj_name != ori_obj_name:
        scenefile['OBJECTS'][obj_name] = scenefile['OBJECTS'][ori_obj_name]
        scenefile['OBJECTS'][obj_name]['meshpath'] = meshpath
        del scenefile['OBJECTS'][ori_obj_name]
        print(f"change from {ori_obj_name} to {obj_name}")
        
    ## change lighting y - just duplicate
    # scenefile['LIGHTS']['light00']['position']['y']
    new_list = scenefile['LIGHTS']['light00']['position']['y'][0:25] * num_samples \
                + scenefile['LIGHTS']['light00']['position']['y'][50:75] * num_samples
    scenefile['LIGHTS']['light00']['position']['y'] = new_list
    
    ## change background - just duplicate
    # scenefile['IMAGES']['imageidx']
    new_list_upright = scenefile['IMAGES']['imageidx'][0:25] + scenefile['IMAGES']['imageidx'][50:75]
    new_list = []
    for imageidx in new_list_upright:
        new_list += [imageidx] * num_samples
    scenefile['IMAGES']['imageidx'] = new_list
    
    ## change invert contrast - empty
    # scenefile['LIGHTS']['light00']['position']['y']
    scenefile['OBJECTFILTERS']['invert'] = ['']
    
    ## change rotation z - just duplicate; all 0s
    # scenefile['OBJECTS'][obj_name]['rotationDegrees']['z']
    new_list = scenefile['OBJECTS'][obj_name]['rotationDegrees']['z'][0:25] * num_samples \
                + scenefile['OBJECTS'][obj_name]['rotationDegrees']['z'][50:75] * num_samples
    scenefile['OBJECTS'][obj_name]['rotationDegrees']['z'] = new_list
    
    # change rotation x, y (x_range=[-45, 45], y_range=[-90, 90]) - add noise

    # update x
    new_list_upright = scenefile['OBJECTS'][obj_name]['rotationDegrees']['x'][0:25] + \
                        scenefile['OBJECTS'][obj_name]['rotationDegrees']['x'][50:75]
    new_list = []
    lower_bound = -45
    upper_bound = 45
    sigma = (upper_bound-lower_bound) * 0.05
    for cur_value in new_list_upright:
        aug_array = np.zeros(num_samples)
        for aug_index in range(num_samples):
            # add noise
            new_value = cur_value + np.random.normal(0, sigma)
            while new_value<lower_bound or new_value>upper_bound:
                new_value = cur_value + np.random.normal(0, sigma)
            aug_array[aug_index] = new_value
        new_list += aug_array.tolist()
    scenefile['OBJECTS'][obj_name]['rotationDegrees']['x'] = new_list

    # update y
    new_list_upright = scenefile['OBJECTS'][obj_name]['rotationDegrees']['y'][0:25] + \
                        scenefile['OBJECTS'][obj_name]['rotationDegrees']['y'][50:75]
    new_list = []
    lower_bound = -90
    upper_bound = 90
    sigma = (upper_bound-lower_bound) * 0.05
    for cur_value in new_list_upright:
        aug_array = np.zeros(num_samples)
        for aug_index in range(num_samples):
            # add noise
            new_value = cur_value + np.random.normal(0, sigma)
            while new_value<lower_bound or new_value>upper_bound:
                new_value = cur_value + np.random.normal(0, sigma)
            aug_array[aug_index] = new_value
        new_list += aug_array.tolist()
    scenefile['OBJECTS'][obj_name]['rotationDegrees']['y'] = new_list
    
    # change size (range=[0.2, 0.42]) - add noise
    # scenefile['OBJECTS'][obj_name]['sizeTHREEJS']
    new_list_upright = scenefile['OBJECTS'][obj_name]['sizeInches'][0:25] + \
                        scenefile['OBJECTS'][obj_name]['sizeInches'][50:75]
    new_list = []
    lower_bound = 0.2
    upper_bound = 0.42
    sigma = (upper_bound-lower_bound) * 0.05
    for cur_value in new_list_upright:
        aug_array = np.zeros(num_samples)
        for aug_index in range(num_samples):
            # add noise
            new_value = cur_value + np.random.normal(0, sigma)
            while new_value<lower_bound or new_value>upper_bound:
                new_value = cur_value + np.random.normal(0, sigma)
            aug_array[aug_index] = new_value
        new_list += aug_array.tolist()
    scenefile['OBJECTS'][obj_name]['sizeInches'] = new_list
    
    ## change duration
    scenefile['durationMS'] = [18]
    
    return scenefile

In [ ]:
AppleMesh00_scenefile_IfAppleMesh00 = add_noise(AppleMesh00_scenefile_IfAppleMesh00, 'AppleMesh00', 
                                    "mkturkfiles/scenebags/objectome3d/face/AppleMesh00applefacepotatoheadnormalhires.glb")
AppleMesh00_scenefile_IfAppleMesh01 = add_noise(AppleMesh00_scenefile_IfAppleMesh01, 'AppleMesh01', 
                                    "mkturkfiles/scenebags/objectome3d/face/AppleMesh01applefacepotatoheadnormalhires.glb")
AppleMesh01_scenefile_IfAppleMesh00 = add_noise(AppleMesh01_scenefile_IfAppleMesh00, 'AppleMesh00', 
                                    "mkturkfiles/scenebags/objectome3d/face/AppleMesh00applefacepotatoheadnormalhires.glb")
AppleMesh01_scenefile_IfAppleMesh01 = add_noise(AppleMesh01_scenefile_IfAppleMesh01, 'AppleMesh01', 
                                    "mkturkfiles/scenebags/objectome3d/face/AppleMesh01applefacepotatoheadnormalhires.glb")

In [ ]:
# save scenefile
_path = f'./20230417_vbsl151upright_samples100_noise0.05vs_AppleMesh00_IfAppleMesh00.json'
with open(_path, 'w') as f:
    json.dump(AppleMesh00_scenefile_IfAppleMesh00, f)

_path = f'./20230417_vbsl151upright_samples100_noise0.05vs_AppleMesh00_IfAppleMesh01.json'
with open(_path, 'w') as f:
    json.dump(AppleMesh00_scenefile_IfAppleMesh01, f)
    
_path = f'./20230417_vbsl151upright_samples100_noise0.05vs_AppleMesh01_IfAppleMesh00.json'
with open(_path, 'w') as f:
    json.dump(AppleMesh01_scenefile_IfAppleMesh00, f)
    
_path = f'./20230417_vbsl151upright_samples100_noise0.05vs_AppleMesh01_IfAppleMesh01.json'
with open(_path, 'w') as f:
    json.dump(AppleMesh01_scenefile_IfAppleMesh01, f)

# for Basel i1

In [ ]:
import copy

In [ ]:
def create_new_scenefile(obj_name, meshpath, USE_TEXTURE, USE_GRAYSCALE, SET_ID):
    # read ori scenefile
    _path = './20211011_Var6vbsl_set0_im151_AppleMesh00_dur200ms_lab_updated.json'
    AppleMesh00_scenefile = json.load(open(_path, 'r'))
    _path = './20211011_Var6vbsl_set0_im151_AppleMesh01_dur200ms_lab_updated.json'
    AppleMesh01_scenefile = json.load(open(_path, 'r'))
    
    # copy based on SET_ID
    if SET_ID==0:
        scenefile = copy.deepcopy(AppleMesh00_scenefile)
    elif SET_ID==1:
        scenefile = copy.deepcopy(AppleMesh01_scenefile)
    else:
        assert False
    
    # obj name
    scenefile['OBJECTS'] = {}
    scenefile['OBJECTS'][obj_name] = AppleMesh00_scenefile['OBJECTS']['AppleMesh00']
    # meshpath
    scenefile['OBJECTS'][obj_name]['meshpath'] = meshpath
    
    # add filters
    if USE_GRAYSCALE:
        scenefile['OBJECTFILTERS']['grayscale'] = [100]
    if not USE_TEXTURE:
        scenefile['OBJECTS'][obj_name]['texture'] = 'false'
    
    scenefile['durationMS'] = [200]
    
    # change size
    target_size = 1.67
    cur_size = scenefile['IMAGES']['sizeInches'][0]
    scenefile['IMAGES']['sizeInches'] = [target_size]
    
    cur_size_list = scenefile['OBJECTS'][obj_name]['sizeInches']
    new_size_list = []
    for size in cur_size_list:
        new_size_list.append(size/cur_size*target_size)
    scenefile['OBJECTS'][obj_name]['sizeInches'] = new_size_list
    
    # save scenefile
    prefix = f'{obj_name}_set{SET_ID}'
    if USE_GRAYSCALE:
        prefix += '_grayscale'
    if not USE_TEXTURE:
        prefix += '_notexture'
    _path = f'./Baseli1/20231103_Var6vbsl_im151_dur200ms_{prefix}.json'
    with open(_path, 'w') as f:
        json.dump(scenefile, f)
    
    return scenefile

In [ ]:
def create_new_token_file(obj_name, meshpath, USE_TEXTURE, USE_GRAYSCALE):
    # read scenefile
    _path = './i2_scenefiles/20220202_token_AppleMesh00.json'
    AppleMesh00_scenefile = json.load(open(_path, 'r'))
    # copy 
    scenefile = copy.deepcopy(AppleMesh00_scenefile)
    
    # obj name
    scenefile['OBJECTS'] = {}
    scenefile['OBJECTS'][obj_name] = AppleMesh00_scenefile['OBJECTS']['AppleMesh00']
    # meshpath
    scenefile['OBJECTS'][obj_name]['meshpath'] = meshpath
    
    # add filters
    if USE_GRAYSCALE:
        scenefile['OBJECTFILTERS'] = {}
        scenefile['OBJECTFILTERS']["grayscale"] = [100]
    if not USE_TEXTURE:
        scenefile['OBJECTS'][obj_name]['texture'] = 'false'
    
    # save scenefile
    prefix = f'{obj_name}'
    if USE_GRAYSCALE:
        prefix += '_grayscale'
    if not USE_TEXTURE:
        prefix += '_notexture'
    _path = f'./Baseli1/20231103_token_{prefix}.json'
    with open(_path, 'w') as f:
        json.dump(scenefile, f)
    
    return scenefile

In [ ]:
for obj_name in ['Baselmesh29', 'Baselmesh231', 'Baselmesh450', 'Baselmesh474']:
    for USE_TEXTURE in [True, False]:
        for USE_GRAYSCALE in [True, False]:
            create_new_token_file(obj_name = obj_name, 
                                  meshpath = f'/mkturkfiles/scenebags/objectome3d/Josh/{obj_name}.glb', 
                                  USE_TEXTURE = USE_TEXTURE, 
                                  USE_GRAYSCALE = USE_GRAYSCALE)
            for SET_ID in [0, 1]:
                create_new_scenefile(obj_name = obj_name, 
                                     meshpath = f'/mkturkfiles/scenebags/objectome3d/Josh/{obj_name}.glb', 
                                     USE_TEXTURE = USE_TEXTURE, 
                                     USE_GRAYSCALE = USE_GRAYSCALE,
                                     SET_ID = SET_ID)